# Beginning and Login

In [122]:
from getpass import getpass
import csv
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions

options = EdgeOptions()
options.use_chromium = True
# options.add_argument("--disable-infobars")
driver = Edge(options=options)


C:\Users\Yasin\AppData\Local\Temp\ipykernel_18740\1432473345.py:11: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)


In [123]:
driver.get("https://twitter.com/login")
sleep(3)

In [124]:
email = driver.find_element_by_xpath('//input[@name="text"]')
email.send_keys("yasininal44@gmail.com")
email.send_keys(Keys.RETURN)
sleep(2)

In [125]:
username = driver.find_element_by_xpath('//input[@name="text"]')
username.send_keys("yasininall")
username.send_keys(Keys.RETURN)
sleep(2)

In [126]:
inp_password = getpass("Enter your password: ")

In [127]:
password = driver.find_element_by_xpath('//input[@name="password"]')
password.send_keys(inp_password)
password.send_keys(Keys.RETURN)
sleep(2)

In [128]:
# start with ronaldo profile
driver.get("https://twitter.com/cristiano")
sleep(2)

- Username
- Displayed Name
- Description
- Number of Followers
- Number of Following
- Birthday
- Data that joined Twitter
- Website
- Within the last 50 posts
  - Number of favorites
  - Number of retweets
  - Number of replies

## Necessary Functions

In [129]:
import datetime
   
def get_tweet_data(tweet):
    try:
        tweet_text = tweet.find_element_by_xpath('.//div[@data-testid="tweetText"]').text
        tweet_hash = hash(tweet_text.strip())
    except:
        tweet_hash = None

    try:
        favorites = tweet.find_element_by_xpath('.//div[@data-testid="like"]').text
        if favorites == "": favorites = 0
    except: favorites = 0
    
    try:
        retweet = tweet.find_element_by_xpath('.//div[@data-testid="retweet"]').text
        if retweet == "": retweet = 0
    except: retweet = 0
    
    try:
        reply = tweet.find_element_by_xpath('.//div[@data-testid="reply"]').text
        if reply == '': reply = 0
    except: reply = 0
    
    # get tweet date
    try:
        tweet_date = tweet.find_element_by_xpath('.//time').get_attribute("datetime")
        # convert to datetime object
        tweet_date = datetime.datetime.strptime(tweet_date, '%Y-%m-%dT%H:%M:%S.%fZ')
    except: tweet_date = None
        
    return [tweet_hash, reply, retweet, favorites, tweet_date]

In [130]:
def get_all_tweets(driver):
    tweet_list = []
    # get all tweets
    tweets = driver.find_elements_by_xpath('//article[@data-testid="tweet"] | //div[@data-testid="tweet"]')
    for tweet in tweets:
        data = get_tweet_data(tweet)
        # tweet_id = ''.join(data[0])
        # data.append(tweet_id)
        tweet_list.append(data)
    return tweet_list
        
def get_tweet_count(driver):
    tweets = driver.find_elements_by_xpath('//article[@data-testid="tweet"] | //div[@data-testid="tweet"]')
    return len(tweets)

In [131]:
def get_profile_info(driver):
    # find a href that contains "followers"
    follower = driver.find_element_by_xpath('//a[contains(@href, "/followers")]')
    
    # if the text contains 'K' letter, then it has more than 10000 followers
    if follower.text.find('K') != -1 or follower.text.find('M') != -1:
        # get the username from the link
        username = follower.get_attribute('href').split('/')[3]
        # print("USERNAME=",username)
        
        displayed_name = driver.find_element_by_xpath('//div[@data-testid="UserName"]').text
        displayed_name = displayed_name.split('\n')[1]
        # get the second line of displayed name
        # print("DISP_NAME=",displayed_name)
        
        try:
            # get description by skipping the line starting with '@'
            description = driver.find_element_by_xpath('//div[@data-testid="UserDescription"]').text
        except: description = ""
        
        # get the number of followers
        followers = follower.text.split(' ')[0].split('.')[0]
        # print('FOLLOWERS=',followers)
        
        # get the number of following
        following = driver.find_element_by_xpath('//a[contains(@href, "/following")]').text
        following = following.split(' ')[0].split('.')[0]
        # print('FOLLOWING=',following)
        
        try:
            # get joined date
            joined = driver.find_element_by_xpath('//span[@data-testid="UserJoinDate"]').text
            # get the text after the 'joined'
            joined = joined.split('Joined')[1]
        except: joined = ""
        
        try:
            # get user url with 'UserUrl'
            user_url = driver.find_element_by_xpath('//a[@data-testid="UserUrl"]//span').text
        except: user_url = ""
        
        return [username, displayed_name, description, followers, following, joined, user_url]
    else:
        return None
        
        

In [132]:
import time

def get_following_list(driver):
    last_append = time.time()
    following_list = []
    while True:
        random_users = driver.find_elements_by_xpath('//div[@data-testid="UserCell"]')
        for usr in random_users:
            username = usr.text.split('\n')[1].split('@')[1]
            if username not in following_list:
                following_list.append(username)
                last_append = time.time()
        driver.execute_script("window.scrollBy(0,1500);")
        sleep(1)
        if len(following_list) > 55 or time.time() - last_append > 5:
            print(f"{len(following_list)} random usernames are fetched")
            return following_list
            

## Main Function

In [133]:
import time

tweet_count = 0
accepted_tweets = []
id_list = []
random_user_list = []
last_position = driver.execute_script("return window.pageYOffset;")
# scrolling = True

spent_time_on_user = time.time()

while True:
    profile_info = get_profile_info(driver)
    if profile_info is None or time.time() - spent_time_on_user > 150:
        driver.get(f"https://twitter.com/{random_user_list[-1]}")
        spent_time_on_user = time.time()
        # delete last item of the list
        random_user_list.pop()
    
    if tweet_count >= 50: 
        # save tweets to csv along 
        with open(f'tweets/{profile_info[0]}.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerows(accepted_tweets) 
        tweet_count = 0
        sleep(4)
        if len(random_user_list) == 0:
            # go following list page that includes "/following" text
            driver.get(f"https://twitter.com/{profile_info[0]}/following")
            random_user_list = get_following_list(driver)
        else:
            driver.get(f"https://twitter.com/{random_user_list[-1]}")
            sleep(5)
            spent_time_on_user = time.time()
            # delete last item of the list
            random_user_list.pop()
            
    tweets = get_all_tweets(driver)
    
    for tw in tweets:
        if tw[0] not in id_list:
            id_list.append(tw[0])
            tweet_count += 1
            accepted_tweets.append(tw)  
            
    scroll_attempt = 0
    # scroll down "a little bit" to load more tweets
    driver.execute_script("window.scrollBy(0,1500);")
    sleep(1)
    current_position = driver.execute_script("return window.pageYOffset;")
    if current_position == last_position:
        scroll_attempt += 1
        if scroll_attempt >= 4:
            # scrolling = False
            break
        else:
            sleep(2)
            
    

58 random usernames are fetched


In [ ]:
import glob
import os

def profile_info_to_csv(profile_info):
    profile_list = []
    # iterate through the csv files in profiles folder
    for file in glob.glob(os.path.join('tweets', '*.csv')):
        # read filename before the .csv
        username = file.split('\\')[1].split('.')[0]
        driver.get(f"https://twitter.com/{username}")
        sleep(4)
        profile_info = get_profile_info(driver)
        if profile_info is None:
            print("profile info is None")
            continue
        profile_list.append(profile_info)
        
    with open(f'profile_info.csv', 'a', encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(profile_list)

In [38]:
with open(f'profile_info.csv', 'a', encoding="utf-8") as f:
    header = ['username', 'displayed_name', 'description', 'followers', 'following', 'joined', 'user_url']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(profile_list)

# Data Analysis

In [64]:
import pandas as pd

# get all the csv files in the tweets folder and concatenate them into one dataframe
# skip empty lines
header = ['tweet_hash', 'reply', 'retweet', 'favorites', 'tweet_date']
df = pd.DataFrame()
for file in glob.glob(os.path.join('tweets', '*.csv')):
    file = pd.read_csv(file, skip_blank_lines=True, encoding="utf-8", names=header)
    df = pd.concat([df, file], ignore_index=True)


In [65]:
df.head(3)

,tweet_hash,reply,retweet,favorites,tweet_date
0,0.000000e+00,"2,092","6,748",129K,2022-05-31 18:17:23
1,8.792970e+18,"1,951","2,727",33.4K,2022-05-28 18:00:00
2,-7.071670e+18,"8,890",29.1K,623.7K,2022-05-16 19:27:16


In [66]:
# number of rows
df.shape[0]

36493

In [79]:
# turn tweet_date into datetime
df['tweet_date'] = pd.to_datetime(df['tweet_date'])
# calculate number of tweets per day
# and save it as a different dataframe
# calculate tweets, replies, retweets, favorites seperately
df['tweet_date'] = pd.to_datetime(df['tweet_date'])
df_per_day = df.groupby(df['tweet_date'].dt.date).count()
df_per_day.tweet_hash

tweet_date
2014-12-22       7
2014-12-24       7
2015-01-13       7
2015-01-17       7
2015-01-20       7
              ... 
2022-05-27     543
2022-05-28     679
2022-05-29     607
2022-05-30    1070
2022-05-31    1313
Name: tweet_hash, Length: 610, dtype: int64

In [80]:
# calculate number of tweets per week
df_per_week = df.groupby(df['tweet_date'].dt.week).count()
df_per_week.tweet_hash

C:\Users\Yasin\AppData\Local\Temp\ipykernel_18740\2271000905.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_per_week = df.groupby(df['tweet_date'].dt.week).count()


tweet_date
1      946
2      833
3     1150
4      682
5      548
6      532
7      450
8      362
9      367
10     490
11     507
12     373
13     546
14     675
15    1174
16    1745
17    1208
18    2681
19    2230
20    2895
21    5346
22    2681
23     239
24     167
25      76
26     297
27     163
28      93
29      96
30     116
31      69
32     146
33      41
34      69
35     108
36      56
37     104
38     109
39     126
40      68
41     206
42     121
43     165
44     373
45     447
46     284
47     284
48     562
49     890
50     859
51     812
52     802
53      76
Name: tweet_hash, dtype: int64

In [81]:
# calculate number of tweets per year and month
df_per_month = df.groupby(df['tweet_date'].dt.month).count()
df_per_month.tweet_hash

tweet_date
1      3801
2      1917
3      2109
4      4802
5     15799
6       798
7       619
8       330
9       459
10      614
11     1604
12     3593
Name: tweet_hash, dtype: int64

In [86]:
# calculate mean of tweets per day, week and month
df_per_day.tweet_hash.mean(), df_per_week.tweet_hash.mean(), df_per_month.tweet_hash.mean()

(59.74590163934426, 687.6415094339623, 3037.0833333333335)

In [87]:
# calculate median of tweets per day, week and month
df_per_day.tweet_hash.median(), df_per_week.tweet_hash.median(), df_per_month.tweet_hash.median()

(24.0, 373.0, 1760.5)

***I think median value would represent a profile better since it is much more resistant to outlier values.***